In [66]:
from utils import *

In [67]:
from azureml.core import Workspace

# Configure experiment
ws = Workspace.from_config()

In [68]:
# Create or get training cluster
aml_cluster = get_aml_cluster(ws, cluster_name="cpu-cluster")
aml_cluster.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [69]:
# Create a run configuration
run_conf = get_run_config(['numpy', 'pandas', 'scikit-learn', 'tensorflow'])

In [70]:
from azureml.core import Dataset

dataset = Dataset.get_by_name(ws, name='titanic')
data_in = dataset.as_named_input('titanic')

In [71]:
from azureml.core import Datastore
from azureml.pipeline.core import PipelineData

datastore = Datastore.get(ws, datastore_name="mldata")
data_train = PipelineData('train', datastore=datastore)
data_test = PipelineData('test', datastore=datastore)

In [72]:
from azureml.data import OutputFileDatasetConfig

data_out = OutputFileDatasetConfig(name="predictions", destination=(datastore, 'titanic/predictions'))
data_out = data_out.read_delimited_files().register_on_complete('titanic.pred')

In [73]:
from azureml.pipeline.steps import PythonScriptStep

step_1 = PythonScriptStep(name='Preprocessing',
                          script_name="preprocess_output.py",
                          source_directory="code",
                          arguments=[
                              "--input", data_in,
                              "--out-train", data_train,
                              "--out-test", data_test],
                          inputs=[data_in],
                          outputs=[data_train, data_test],
                          runconfig=run_conf,
                          compute_target=aml_cluster)


In [74]:
from azureml.pipeline.steps import PythonScriptStep

step_2 = PythonScriptStep(name='Training',
                          script_name="train_output.py",
                          source_directory="code",
                          arguments=[
                              "--in-train", data_train,
                              "--in-test", data_test,
                              "--output", data_out],
                          inputs=[data_train, data_test],
                          outputs=[data_out],
                          runconfig=run_conf,
                          compute_target=aml_cluster)

In [75]:
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(ws, steps=[step_1, step_2])

In [76]:
pipeline.validate()

[]

In [77]:
from azureml.core import Experiment

exp = Experiment(ws, "azureml-pipeline")
run = exp.submit(pipeline)

Created step Preprocessing [ba122dc3][debaed5b-722b-4a15-b19f-3b06d1615d90], (This step is eligible to reuse a previous run's output)Created step Training [051abdfe][fe1f0d98-4f6b-4a54-a10f-8f5f1896ba36], (This step is eligible to reuse a previous run's output)



ClientRequestError: Error occurred in request., ConnectionError: HTTPSConnectionPool(host='westeurope.api.azureml.ms', port=443): Max retries exceeded with url: /pipelines/v1.0/subscriptions/21dc412b-d9eb-42e7-8317-55bc8eb10cf5/resourceGroups/packt/providers/Microsoft.MachineLearningServices/workspaces/mldemows/PipelineRuns/Submit/d78cfd21-a872-4f2b-8a91-60c96656cb15 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x13b9237f0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …